<a href="https://colab.research.google.com/github/markusdiekmann95/Morph-U-Net/blob/main/Notebooks/first_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install git+https://github.com/online-ml/river --upgrade

  Cloning https://github.com/online-ml/river to /tmp/pip-req-build-88uztpum
  Running command git clone -q https://github.com/online-ml/river /tmp/pip-req-build-88uztpum
  Created wheel for river: filename=river-0.10.1-cp37-cp37m-linux_x86_64.whl size=1658784 sha256=d5332d079d30c31696250040ecf14c27967f9ca6990d65bdecf5b116581d968d
  Stored in directory: /tmp/pip-ephem-wheel-cache-0sn_qmf5/wheels/a1/81/b6/35dc3f68cb6c44cc3967a6a2f40410150d6ed5df2ff2b83f41
Successfully built river


In [ ]:
import pandas as pd
import river
import numpy as np
from tqdm.notebook import tqdm

In [ ]:
df = pd.read_csv("https://raw.githubusercontent.com/Mergim01/RiverStreamClassification/main/ClimateGPT.csv")
# Convert to Format
df.to_dict()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40999 entries, 0 to 40998
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  40999 non-null  object 
 1   id          40997 non-null  object 
 2   timestamp   40997 non-null  object 
 3   text        40992 non-null  object 
 4   user        40995 non-null  object 
 5   artificial  40995 non-null  float64
dtypes: float64(1), object(5)
memory usage: 1.9+ MB


In [ ]:
df = df[["text","artificial"]]
df = df.dropna() #let us first remove all rows with NaNs
df[["artificial"]] = df[["artificial"]].applymap(np.int64)

from sklearn.model_selection import train_test_split

train, test = train_test_split(df, test_size=0.2)
#mergim ist schlau

In [ ]:
# Convert to Tuple
train_set = train.to_records(index=False)
test_set = test.to_records(index=False)
train_set

rec.array([('Biden bill on social welfare, climate change estimated to kill 103K jobs, raise taxes on 80% of U.S. - https://t.co/I5ZxmX4Y1q - @0317a13c06b2533b6e72', 1),
           ('‘Whitney’ Director #KevinMacdonald Behind #ITV Doc On UK Floods, Broadcaster To Unveil Climate Action Plans https://t.co/18BqScDf88 https://t.co/bN81ZZk0r6', 0),
           ('🌱 #COP26 starts in 2 weeks time. Please join me and sign the #Youth4Pacific Declaration on Climate Action in the Pacific.   https://t.co/uJfT2k47tT  There is no time left to waste.', 1),
           ...,
           ('@c7f8ea53ebe68896bb32 Wait till bhakts start objecting on burning paper and raising issues of air pollution and temperature change', 0),
           ('@1d5e06bd1c905cb6d182 @60f0feeb01f24290cede @747d0798bae1c855fd5d And we will be the reason for air pollution.', 1),
           ('@536f14c08dfcc37742ac Not a single person right to vote is in danger. Like everything, the climate has been politicized and division created, that

In [ ]:
from river.naive_bayes import MultinomialNB
from river.neighbors import KNNClassifier
from river.tree import HoeffdingTreeClassifier, ExtremelyFastDecisionTreeClassifier
from river import linear_model
from river.feature_extraction import BagOfWords,TFIDF
from river import preprocessing
from river.compose import Pipeline 
from river import optim

In [ ]:
# Build Pipeline
pipe_nb = Pipeline(('vectorizer',BagOfWords(lowercase=True)),('nb',MultinomialNB()))
pipe_nb

Pipeline (
  BagOfWords (
    on=None
    strip_accents=True
    lowercase=True
    preprocessor=None
    tokenizer=<built-in method findall of re.Pattern object at 0x7f4721173030>
    ngram_range=(1, 1)
  ),
  MultinomialNB (
    alpha=1.
  )
)

In [ ]:
# Specify Steps
pipe_nb.steps

OrderedDict([('vectorizer', BagOfWords (
                on=None
                strip_accents=True
                lowercase=True
                preprocessor=None
                tokenizer=<built-in method findall of re.Pattern object at 0x7f4721173030>
                ngram_range=(1, 1)
              )), ('nb', MultinomialNB (
                alpha=1.
              ))])

In [ ]:
# Train
for text,label in tqdm(train_set):
    pipe_nb = pipe_nb.learn_one(text,label)

  0%|          | 0/32792 [00:00<?, ?it/s]

In [ ]:
# Make a Prediction
test = 'Mergim ist eine Maschine'
pred = pipe_nb.predict_one(test)
# Prediction Prob
proba_ = pipe_nb.predict_proba_one(test)
print('Predicted test sentence as',pred,' and probability of classes for test as',proba_)

Predicted test sentence as 1  and probability of classes for test as {1: 0.8423078235287216, 0: 0.15769217647127906}


In [ ]:
test_set[0][0]

'Prince William’s Earthshot Award honors Costa Rica and Milan in Climate Fight –\xa0CityNewss https://t.co/KZCLQGdchM'

In [ ]:
example = test_set[0]
pred = pipe_nb.predict_one(example[0])
# Prediction Prob
proba_ = pipe_nb.predict_proba_one(example[0])
print('sentence:',example[0])
print('Predicted test sentence as',pred,' and probability of classes for test as',proba_)
print('True Label:',example[1])

sentence: Prince William’s Earthshot Award honors Costa Rica and Milan in Climate Fight – CityNewss https://t.co/KZCLQGdchM
Predicted test sentence as 0  and probability of classes for test as {1: 5.988650917124152e-05, 0: 0.9999401134908233}
True Label: 0


In [ ]:
# Update the Model on the test data & Check Accuracy
metric = river.metrics.F1()
metrics = []
for text,label in tqdm(test_set):
    y_pred_before = pipe_nb.predict_one(text)
    metric = metric.update(label,y_pred_before)
    metrics.append(metric)
    # Has already learnt the pattern
    #pipe_nb = pipe_nb.learn_one(text,label)
print(metric)

  0%|          | 0/8199 [00:00<?, ?it/s]

F1: 73.41%


In [ ]:
metrics

[F1: 73.41%,
 F1: 73.41%,
 F1: 73.41%,
 F1: 73.41%,
 F1: 73.41%,
 F1: 73.41%,
 F1: 73.41%,
 F1: 73.41%,
 F1: 73.41%,
 F1: 73.41%,
 F1: 73.41%,
 F1: 73.41%,
 F1: 73.41%,
 F1: 73.41%,
 F1: 73.41%,
 F1: 73.41%,
 F1: 73.41%,
 F1: 73.41%,
 F1: 73.41%,
 F1: 73.41%,
 F1: 73.41%,
 F1: 73.41%,
 F1: 73.41%,
 F1: 73.41%,
 F1: 73.41%,
 F1: 73.41%,
 F1: 73.41%,
 F1: 73.41%,
 F1: 73.41%,
 F1: 73.41%,
 F1: 73.41%,
 F1: 73.41%,
 F1: 73.41%,
 F1: 73.41%,
 F1: 73.41%,
 F1: 73.41%,
 F1: 73.41%,
 F1: 73.41%,
 F1: 73.41%,
 F1: 73.41%,
 F1: 73.41%,
 F1: 73.41%,
 F1: 73.41%,
 F1: 73.41%,
 F1: 73.41%,
 F1: 73.41%,
 F1: 73.41%,
 F1: 73.41%,
 F1: 73.41%,
 F1: 73.41%,
 F1: 73.41%,
 F1: 73.41%,
 F1: 73.41%,
 F1: 73.41%,
 F1: 73.41%,
 F1: 73.41%,
 F1: 73.41%,
 F1: 73.41%,
 F1: 73.41%,
 F1: 73.41%,
 F1: 73.41%,
 F1: 73.41%,
 F1: 73.41%,
 F1: 73.41%,
 F1: 73.41%,
 F1: 73.41%,
 F1: 73.41%,
 F1: 73.41%,
 F1: 73.41%,
 F1: 73.41%,
 F1: 73.41%,
 F1: 73.41%,
 F1: 73.41%,
 F1: 73.41%,
 F1: 73.41%,
 F1: 73.41%,
 F1: 73.41%,